In [127]:
%load_ext autoreload
%autoreload 2
from courtvision.trackers import Tracker, StateIdx

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
# Test
from torch.testing import assert_close

tracker = Tracker(num_particles=1000)
assert all(tracker.states[:, StateIdx.weight] > 0.0)
tracker.states[:, 9] = Tracker.normalized_weights(states=tracker.states)

assert_close(tracker.states.select("state", 9).sum().item(), 1.0, atol=1e-3, rtol=1e-3)

In [129]:
# Test set state
import torch

tracker = Tracker(num_particles=1000)
tracker.set_states_to(torch.tensor([1.0, 2.0, 3.0]))
assert_close(tracker.states[:, StateIdx.x].mean().item(), 1.0, atol=1e-3, rtol=1e-3)

2

In [98]:
from torch.nn.functional import mse_loss
import torch

# Test:
tracker = Tracker(num_particles=1000)
obs_in_image = torch.tensor([[1.0, 1.0]]).repeat(1000, 1)

# tracker.normalize_weights()
tracker.H = torch.eye(3, 4)
# assert (tracker.states[:,:3] - Tracker.state_to_observation(tracker.states, tracker.H)).sum() == 0.0
assert Tracker.state_to_observation(tracker.states, tracker.H).shape == torch.Size(
    [1000, 2]
)
pred_image_states = Tracker.state_to_observation(tracker.states, tracker.H)
likelihoods = Tracker.likelihood(obs_state=obs_in_image, pred_state=pred_image_states)
assert all(tracker.states.select("state", 9) > 0.0), "weights should be positive"

assert all(likelihoods > 0.0), "likelihoods should be positive"
tracker.states[:, 9] = Tracker.update_weights(
    tracker.states.select("state", 9), likelihoods
)
assert all(tracker.states.select("state", 9) > 0.0), "weights should be positive"
tracker.states = Tracker.resample(
    tracker.states.rename(None), tracker.states.select("state", 9)
)
# print(likelihoods.shape)

In [99]:
# Define the 3D point in homogeneous coordinates
# P_hom = np.array([[X], [Y], [Z], [1]], dtype=np.float32)

# # Project the 3D point to the image plane using the projection matrix
# P_proj_hom = np.dot(P, P_hom)

# # Normalize the image coordinates
# P_proj = P_proj_hom[:2] / P_proj_hom[2]

# # Print the projected point on the image plane
# print("Projected point:", P_proj)

In [100]:
from courtvision.geometry import corners_world_3d, convert_corners_to_coords

convert_corners_to_coords(corners_world_3d)
from courtvision.geometry import get_corners_image, get_corners_verital_plane_on_image

corners_image = get_corners_image("curated_001/curated_001_frame_0001")
corners_image_other = get_corners_verital_plane_on_image(
    "curated_001/curated_001_frame_0001", plane="both"
)
corners_image.update(corners_image_other)
for k in list(corners_world_3d.keys()):
    if k not in corners_image:
        corners_world_3d.pop(k)


corners_image.keys(), corners_world_3d.keys()

(dict_keys(['a_front_left', 'b_front_right', 'c_back_left', 'd_back_right', 'e_left_near_serve_line', 'f_right_near_serve_line', 'g_left_far_serve_line', 'h_right_far_serve_line', 'm_top_front_left', 'n_top_front_right', 'o_top_back_left', 'p_top_back_right']),
 dict_keys(['a_front_left', 'b_front_right', 'c_back_left', 'd_back_right', 'e_left_near_serve_line', 'f_right_near_serve_line', 'g_left_far_serve_line', 'h_right_far_serve_line', 'm_top_front_left', 'n_top_front_right', 'o_top_back_left', 'p_top_back_right']))

In [159]:
import torch
import kornia
from courtvision.geometry import corners_world_3d

#  REF https://ksimek.github.io/2013/08/13/intrinsic/
world_points = torch.tensor(
    convert_corners_to_coords(corners_world_3d), dtype=torch.float64
).unsqueeze(0)

img_points = torch.tensor(
    convert_corners_to_coords(corners_image), dtype=torch.float64
).unsqueeze(0)
print(f"{img_points=}")
from kornia.geometry.conversions import (
    normalize_pixel_coordinates,
    denormalize_pixel_coordinates,
    normalize_pixel_coordinates3d,
)

img_points_N = normalize_pixel_coordinates(img_points, width=1280, height=720)
img_points_d = denormalize_pixel_coordinates(img_points_N, width=1280, height=720)
print(f"{img_points_d=}")
fx = 800
fy = 800
cy = 720 / 2

cx = 1280 / 2


intrinsics = torch.tensor(
    [
        [
            [fx, 0.0, cx],
            [0.0, fy, cy],
            [0.0, 0.0, 1.0],
        ]
    ],
    dtype=torch.float64,
)

print(world_points.shape, img_points.shape, intrinsics.shape)
# torch.Size([1, 6, 3]) torch.Size([1, 6, 2]) torch.Size([1, 3, 3])
world_points_N = normalize_pixel_coordinates3d(
    world_points, depth=2000.0, height=300.0, width=1000.0
)
# Solve PnP
import cv2
import numpy as np

dist_coeffs = np.array([0, 0, 0, 0, 0], dtype=np.float32)

success, rvec, tvec = cv2.solvePnP(
    world_points_N.numpy(),
    img_points_N.numpy(),
    intrinsics.squeeze(0).numpy(),
    dist_coeffs,
    flags=cv2.SOLVEPNP_UPNP,
)
print(f"{success=} {rvec=} {tvec=}")
pred_world_to_cam = kornia.geometry.solve_pnp_dlt(
    world_points_N, img_points_N, intrinsics, svd_eps=0.0000000001
)

print(pred_world_to_cam.shape)
# torch.Size([1, 3, 4])

pred_world_to_cam
world_points_N

img_points=tensor([[[0.1111, 0.8790],
         [0.8944, 0.8864],
         [0.3119, 0.2900],
         [0.6865, 0.2894],
         [0.1661, 0.7259],
         [0.8415, 0.7278],
         [0.2964, 0.3436],
         [0.7097, 0.3439],
         [0.0878, 0.4894],
         [0.9141, 0.4885],
         [0.3067, 0.0870],
         [0.6949, 0.6949]]], dtype=torch.float64)
img_points_d=tensor([[[0.1111, 0.8790],
         [0.8944, 0.8864],
         [0.3119, 0.2900],
         [0.6865, 0.2894],
         [0.1661, 0.7259],
         [0.8415, 0.7278],
         [0.2964, 0.3436],
         [0.7097, 0.3439],
         [0.0878, 0.4894],
         [0.9141, 0.4885],
         [0.3067, 0.0870],
         [0.6949, 0.6949]]], dtype=torch.float64)
torch.Size([1, 12, 3]) torch.Size([1, 12, 2]) torch.Size([1, 3, 3])
success=True rvec=array([[ 1.73129197],
       [-0.4755916 ],
       [ 1.13913989]]) tvec=array([[-1939726.92634732],
       [-1090176.10143233],
       [ 2426190.30146557]])
torch.Size([1, 3, 4])


tensor([[[-1.0000e+00, -1.0000e+00, -1.0000e+00],
         [ 5.0025e-04, -1.0000e+00, -1.0000e+00],
         [-1.0000e+00,  3.0040e+00, -1.0000e+00],
         [ 5.0025e-04,  3.0040e+00, -1.0000e+00],
         [-1.0000e+00, -5.9960e-01, -1.0000e+00],
         [ 5.0025e-04, -5.9960e-01, -1.0000e+00],
         [-1.0000e+00,  2.6036e+00, -1.0000e+00],
         [ 5.0025e-04,  2.6036e+00, -1.0000e+00],
         [-1.0000e+00, -1.0000e+00,  1.0067e+00],
         [ 5.0025e-04, -1.0000e+00,  1.0067e+00],
         [-1.0000e+00,  3.0040e+00,  1.0067e+00],
         [ 5.0025e-04,  3.0040e+00,  1.0067e+00]]], dtype=torch.float64)

In [155]:
intrinsics.unsqueeze(0).shape

torch.Size([1, 1, 3, 3])

In [160]:
tvec, rvec

(array([[-1939726.92634732],
        [-1090176.10143233],
        [ 2426190.30146557]]),
 array([[ 1.73129197],
        [-0.4755916 ],
        [ 1.13913989]]))

In [143]:
pred_world_to_cam.shape

torch.Size([1, 3, 4])

In [136]:
tracker = Tracker(num_particles=1000, world_to_cam=pred_world_to_cam)

In [137]:
tracker.H.to(dtype=torch.float32)

tensor([[[-5.8979e-01,  7.1178e-01, -3.8146e-01,  9.6423e+02],
         [-3.3216e-01,  2.1675e-01,  9.1798e-01,  5.4303e+02],
         [ 7.3609e-01,  6.6812e-01,  1.0859e-01, -1.2034e+03]]])

In [148]:
obs_in_image = torch.tensor([[1.0, 1.0]]).repeat(1000, 1)
[0.8944, 0.8864],
from kornia.geometry.conversions import normalize_homography3d

tracker.set_states_to(torch.tensor([-1.0, -1.0, -1.0]))
dd = Tracker.state_to_observation(
    tracker.states, tracker.H.to(dtype=torch.float32).squeeze(0)
)
dd, img_points_N

(tensor([[-0.8005, -0.4500],
         [-0.8005, -0.4500],
         [-0.8005, -0.4500],
         ...,
         [-0.8005, -0.4500],
         [-0.8005, -0.4500],
         [-0.8005, -0.4500]]),
 tensor([[[-0.9998, -0.9976],
          [-0.9986, -0.9975],
          [-0.9995, -0.9992],
          [-0.9989, -0.9992],
          [-0.9997, -0.9980],
          [-0.9987, -0.9980],
          [-0.9995, -0.9990],
          [-0.9989, -0.9990],
          [-0.9999, -0.9986],
          [-0.9986, -0.9986],
          [-0.9995, -0.9998],
          [-0.9989, -0.9981]]], dtype=torch.float64))

In [145]:
dd

tensor([[-0.8012, -0.4512],
        [-0.8012, -0.4512],
        [-0.8012, -0.4512],
        ...,
        [-0.8012, -0.4512],
        [-0.8012, -0.4512],
        [-0.8012, -0.4512]])

In [109]:
img_points.min()

tensor(0.0870, dtype=torch.float64)

In [150]:
import cv2
import numpy as np

# 3D object points in world coordinates
object_points = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]], dtype=np.float32)

# 2D image points in image coordinates
image_points = np.array([[10, 10], [20, 10], [10, 20], [20, 20]], dtype=np.float32)

# Camera intrinsic matrix (fx, fy, cx, cy)
camera_matrix = np.array([[500, 0, 320], [0, 500, 240], [0, 0, 1]], dtype=np.float32)

# Distortion coefficients (k1, k2, p1, p2, k3)
dist_coeffs = np.array([0, 0, 0, 0, 0], dtype=np.float32)

# Solve PnP
success, rvec, tvec = cv2.solvePnP(
    object_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP
)
print(f"{success=} {rvec=} {tvec=}")

success=True rvec=array([[0.],
       [0.],
       [0.]]) tvec=array([[-31.75634152],
       [-23.55795647],
       [ 51.23990293]])
